In [ ]:
import os

import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
from scipy import special as spf

import SSLimPy.interface.sslimpy as sslimpy
import SSLimPy.LIMsurvey.Covariance as sco
from SSLimPy.LIMsurvey import PowerSpectra
from SSLimPy.LIMsurvey import higherorder as sho
from SSLimPy.utils import utils as utl

In [ ]:
envkey = "OMP_NUM_THREADS"
# Set this environment variable to the number of available cores in your machine,
# to get a fast execution of the Einstein Boltzmann Solver
print("The value of {:s} is: ".format(envkey), os.environ.get(envkey))
os.environ[envkey] = str(12)
print("The value of {:s} is: ".format(envkey), os.environ.get(envkey))

In [ ]:
cosmodict = {"h": 0.68, "Omegam": 0.3, "Omegab": 0.04, "As": 2.1e-9, "mnu": 0.06}
settings = {
    "code": "class",
    "do_RSD": True,
    "nonlinearRSD": True,
    "FoG_damp": "ISTF_like",
    "halo_model_PS": True,
    "Smooth_window": False,
    "kmin": 5e-3 * u.Mpc**-1,
    "kmax": 10 * u.Mpc**-1,
    "nk": 100,
}

# Deactivate LIM for testing
astrodict = {
    "model_type":"ML",
    "model_name":"MassPow",
    "model_par": {
        "A":1,
        "b":0
    },
    "Mmin" : 1e3 * u.Msun,
    "Mmax" : 1e16 * u.Msun,
    "nM" : 256,
}
Asslimpy = sslimpy.sslimpy(settings_dict=settings, cosmopars=cosmodict, astropars=astrodict)

pobs = PowerSpectra.PowerSpectra(Asslimpy.fiducialcosmo, Asslimpy.fiducialastro)

In [ ]:
M = pobs.astro.M
z = 2

b1 = pobs.astro.bias_coevolution.b1(M, z, 1.68)
b2 = pobs.astro.bias_coevolution.b2(M, z, 1.68)
b3 = pobs.astro.bias_coevolution.b3(M, z, 1.68)

In [ ]:
plt.semilogx(M, b1, label="$b_1$")
plt.semilogx(M, b2, label="$b_2$")
plt.semilogx(M, b3/6, label="$b_3/6$")
plt.xlabel(r"$M \:[M_\odot]$")
plt.ylabel("halo bias")
plt.legend()
plt.ylim((-3,5))


In [ ]:
dndM = pobs.astro.bias_coevolution.sc_hmf(M, z, 1.68)
#dndM.shape
plt.loglog(M,M[:,None] * b1[:, None] * dndM, c="blue", label="$b_1$")
plt.loglog(M,M[:,None] * b2[:, None] * dndM, c="orange", label="$b_2$")
plt.loglog(M,M[:,None] * -b2[:, None] * dndM, ls="--", c="orange")
plt.loglog(M,M[:,None] * (b3/6)[:, None] * dndM, c="green", label="$b_3/6$")
plt.loglog(M,M[:,None] * -(b3/6)[:, None] * dndM, ls="--",c="green")
plt.legend()
plt.xlabel(r"$M$ $[M_\odot]$")
plt.ylabel(r"M $\times$ halo bias $\times$ HMF $[\mathrm{Mpc}^{-3}]$")


In [ ]:
dndM = pobs.astro.bias_coevolution.sc_hmf(M, [0, 1, 2], 1.686).to(u.Msun**-1 * u.Mpc**-3)
dndM_og = pobs.astro.halomassfunction(M, [0, 1, 2]).to(u.Msun**-1 * u.Mpc**-3)
fig, axs = plt.subplots(2, 1, sharex=True, height_ratios=[3,1])
ca = ["blue", "orange", "green"]
for ic, ci in enumerate(ca):
    axs[0].loglog(M, M * dndM[:,ic], c=ci, ls="-")
    axs[0].loglog(M, M * dndM_og[:,ic], c=ci, ls="--")
    axs[1].semilogx(M, (dndM/dndM_og-1).to(1)[:,ic]*100, color=ci)
fig.subplots_adjust(hspace=0)
axs[0].plot([],[], c=ca[0], label="$z$=0")
axs[0].plot([],[], c=ca[1], label="$z$=1")
axs[0].plot([],[], c=ca[2], label="$z$=2")
axs[1].set_xlabel(r"$M$ $[M_\odot]$")
axs[0].set_ylabel(r"$M \times$ HMF $[\mathrm{Mpc}^{-3}]$")
axs[1].set_ylabel(r"% rel deviation")

axs[0].legend(ncol=3)

In [ ]:
sigmaM = pobs.astro.sigmaM(M,z)
plt.loglog(M,(1.6865/sigmaM)**2)
plt.xlabel(r"$M \:[M_\odot]$")
plt.ylabel(r"Colapsefraction $\nu=(\delta_{\rm c}/\sigma(M))^2$")

In [ ]:
R = np.geomspace(1e-4, 1e3)*pobs.astro.Mpch
sigmaR = pobs.cosmology.sigmaR_of_z(R, z, tracer="clustering")

In [ ]:
plt.loglog(R, sigmaR)

In [ ]:
nuM =  (1.6865/sigmaM)**2
nu = np.geomspace(min(nuM), max(nuM),num=256)
Fdv = pobs.astro.bias_coevolution.unnorm_collapsefunction(nu)
plt.loglog(nu,Fdv)
1/np.trapz(Fdv, nu)

In [ ]:
k = pobs.k
ff = pobs.ft_NFW(k, M, 0)
dndM = pobs.astro.bias_coevolution.sc_hmf(M,z,1.6865)[:,0]

plt.loglog(k, ff**2, c="red", alpha=0.1)
plt.loglog(k, ff, "-k", alpha=0.1)M
¯ρ 
plt.xlabel(r"$k [\mathrm{Mpc}^{-1}]$")
plt.ylabel("Fourier Transformed NFW")
plt.loglog([],[],c="black", label="Halo")
plt.loglog([],[],c="red", label="Halo^2")
plt.legend()

"okay"

In [ ]:
dndM.shape

In [ ]:


plt.loglog(k,ff[:, 0],label="lowest mass")
plt.loglog(k,ff[:, -1],label="highest mass")
plt.legend()

In [ ]:
z = 0
k = pobs.k
mu, wi = spf.roots_legendre(9)

In [ ]:
Pk = pobs.cosmology.matpow(k, z, tracer="clustering").to(u.Mpc**3)
f = pobs.cosmology.growth_rate(1e-3 * u.Mpc**-1, z, tracer="clustering")
q = (k[1:]+k[:-1])/2

Lmb1 = pobs.astro.bavghalo("b1", z, 1)
L2mb1 = pobs.astro.bavghalo("b1", z, 2)

rsP = sho.PowerSpectrumLO(Lmb1.item(), L2mb1.item(), f.item(), q, mu, 0, k, Pk.value)

In [ ]:
I01 = pobs.reduced_halo_temperature_moments(z, q, mu, moment=1).to(u.uK)
I01 = I01[:,0,0]
I02 = pobs.reduced_halo_temperature_moments(z, q, q, mu, mu,moment=2).to(u.uK**2 * u.Mpc**3)
uI02 = I02[0,0,0,0,0].unit
a = [I02[iq, iq, 0,0 ,0].value for iq in range(len(q))]
I02 = (np.array(a) * uI02)

P = I01[:, None]**2 * rsP*u.Mpc**3 + I02[:, None]

In [ ]:
P0 = np.sum(P* wi * spf.legendre(0)(mu),axis=-1)
P2 = np.sum(P* wi * spf.legendre(2)(mu),axis=-1)
P4 = np.sum(P* wi * spf.legendre(4)(mu),axis=-1)

In [ ]:
plt.loglog(q, q * P0, label="Monopol")
plt.loglog(q, q * P2, label="Quadrupole")
plt.loglog(q, q * P4, label="Hexadecipol")
plt.xlabel(r"$k\:[\mathrm{Mpc}^{-1}]$")
plt.ylabel(r"$k\,P_\ell(k)\:[\mathrm{Mpc}^{2}]$")
plt.legend()
plt.loglog(k,k*Pk*pobs.astro.CLT(z)**2, ls=":", c="black")

In [ ]:
#compare to [0506395]
Mtest = (np.geomspace(1e10,1e16)*pobs.astro.Msunh).to(u.Msun)
logMt = np.log(Mtest.value)
dndlnM = (pobs.astro.bias_coevolution.sc_hmf(M,z,1.6865)[:,0] * M).to(u.Mpc**-3)

# power-law extrapolation (not correct the cut off is much steeper)
logM = np.log((M.to(u.Msun).value))
logy = np.log(dndlnM.value)
y = np.exp(utl.linear_interpolate(logM, logy, logMt))*u.Mpc**-3
y = y.to(pobs.astro.Mpch**-3)

In [ ]:
plt.loglog(Mtest.to(pobs.astro.Msunh),y)
plt.xlim(1e10,1e16)
plt.grid(visible=True)
plt.xlabel(r"$M \:[h^{-1} M_\odot]$")
plt.ylabel(r"$\mathrm{d}n/\mathrm{dln}M\: [h^3 \mathrm{Mpc}^{-3}]$")

In [ ]:
#direct computation of the shotnoise
Psh = np.trapz(dndM[:]* M, np.log(M.value)) * pobs.astro.CLT(z)**2
Psh

In [ ]:
plt.loglog(q, P0, label="Monopol")
plt.loglog(q, P2, label="Quadrupole")
plt.loglog(q, P4, label="Hexadecipol")
plt.xlabel(r"$k\:[\mathrm{Mpc}^{-1}]$")
plt.ylabel(r"$k\,P_\ell(k)\:[\mathrm{Mpc}^{2}]$")
plt.legend()
plt.loglog(k,k*Pk*pobs.astro.CLT(z)**2, ls=":", c="black")
plt.loglog(k,k*Psh, ls="--", c="black")